# UnQTube2 Setup Guide

This notebook will guide you through the process of setting up [UnQTube2](https://github.com/Sandeepgaddam5432/UnQTube2).

## 1. Clone Repository and Install Dependencies

First, we'll clone the repository from GitHub and install all required packages:

In [ ]:
!git clone https://github.com/Sandeepgaddam5432/UnQTube2.git
%cd UnQTube2
!pip install -q -r requirements.txt

## 2. Configure Cloudflare Tunnel for Remote Access

We'll use Cloudflare Tunnel to create a secure tunnel to expose our local Streamlit server to the internet.
This will give you a public URL to access your UnQTube2 application from anywhere.

In [ ]:
import os
import platform
import subprocess

# Determine the system architecture and download the appropriate cloudflared binary
system = platform.system().lower()
machine = platform.machine().lower()

cloudflared_url = ""
if system == "linux":
    if "arm" in machine or "aarch" in machine:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-arm64"
    else:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
elif system == "darwin":
    if "arm" in machine or "aarch" in machine:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-darwin-amd64"
    else:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-darwin-amd64"
else:  # Windows
    cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-windows-amd64.exe"

# Download cloudflared
print(f"📥 Downloading cloudflared from {cloudflared_url}...")
!wget -q {cloudflared_url} -O cloudflared
!chmod +x cloudflared
print("✅ Cloudflared downloaded and made executable!")

## 3. Launch Application and Generate Public URL

Now we'll start the Streamlit server and create a Cloudflare tunnel to make it accessible online:

In [ ]:
import subprocess
import time
import re
import os
import signal
import sys

print("🚀 Starting UnQTube2...")
# Start Streamlit server on port 8501 in the background
streamlit_proc = subprocess.Popen([
    "streamlit", "run", "./webui/Main.py", "--server.port=8501"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Wait for the server to initialize
time.sleep(5)

print("🌐 Creating Cloudflare tunnel to expose UnQTube2...")
# Start cloudflared in the background and redirect output to a log file
cloudflared_log = "cloudflared.log"

# Make sure the log file is empty before we start
if os.path.exists(cloudflared_log):
    open(cloudflared_log, 'w').close()

# Initialize cloudflared_proc variable
cloudflared_proc = None
connection_success = False

try:
    # Start cloudflared with proper error handling
    try:
        cloudflared_proc = subprocess.Popen(
            ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
            stdout=open(cloudflared_log, "w"),
            stderr=subprocess.STDOUT
        )
    except Exception as e:
        print(f"❌ Failed to start cloudflared: {str(e)}")
        print("Please check if cloudflared was downloaded correctly and is executable.")
        raise

    # Monitor the tunnel status with a more robust approach
    print("⏳ Waiting for tunnel to establish (this may take up to 30 seconds)...")
    max_wait_time = 30  # Maximum wait time in seconds
    start_time = time.time()
    public_url = None
    error_found = False

    # Keep checking until we find a URL, find an error, or timeout
    while time.time() - start_time < max_wait_time:
        # Check if cloudflared process is still running
        if cloudflared_proc.poll() is not None:
            print(f"❌ Cloudflared process exited unexpectedly with code {cloudflared_proc.returncode}")
            error_found = True
            break
        
        # Read the current log content
        try:
            with open(cloudflared_log, "r") as f:
                log_content = f.read()
            
            # Priority 1: First check for the success URL
            url_match = re.search(r'https://[\w-]+\.trycloudflare\.com', log_content)
            if url_match:
                public_url = url_match.group(0)
                break
                
            # Priority 2: Only if URL is not found, check for critical errors
            critical_error_patterns = [
                'ERR ', 'CRIT ', 'fail', 'unreachable', 'denied'
            ]
            
            for pattern in critical_error_patterns:
                if re.search(pattern, log_content, re.IGNORECASE):
                    error_line = next((line for line in log_content.split("\n") 
                                      if re.search(pattern, line, re.IGNORECASE)), "")
                    print(f"❌ Critical error detected in cloudflared log: {error_line}")
                    error_found = True
                    break
            
            if error_found:
                break
                
        except Exception as e:
            print(f"Warning: Failed to read log file: {str(e)}")
        
        # Wait before checking again
        time.sleep(1)
        print(".", end="", flush=True)

    print()  # New line after the progress dots

    # Handle the outcome
    if public_url:
        print("✅ Deployment complete! Access your UnQTube2 at:")
        print(f"�� {public_url}")
        
        # Improved liveness check: multiple connection attempts over time
        print("🔍 Verifying connection with multiple checks...")
        verification_attempts = 3
        
        import requests
        for attempt in range(verification_attempts):
            try:
                print(f"  Attempt {attempt+1}/{verification_attempts}...")
                response = requests.head(public_url, timeout=10)
                if response.status_code < 400:
                    print(f"  ✅ Connection verified successfully on attempt {attempt+1}!")
                    connection_success = True
                    break
                else:
                    print(f"  ⚠️ Attempt {attempt+1}: Status code {response.status_code}")
            except Exception as e:
                print(f"  ⚠️ Attempt {attempt+1} failed: {str(e)}")
            
            # Wait between attempts (but not after the last one)
            if attempt < verification_attempts - 1:
                print(f"  Waiting 5 seconds before next attempt...")
                time.sleep(5)
        
        if connection_success:
            print("\n✅ Connection verified! Your UnQTube2 instance is accessible online.")
            print(f"🔗 {public_url}")
            
            # Start persistent loop to keep the notebook cell running
            print("\n⚡ Tunnel is now live and stable! Press the STOP button in Colab to shut down.")
            print("   (This cell will remain running to keep the tunnel alive)")
            
            # Infinite loop to keep processes alive
            try:
                while True:
                    # Check if processes are still alive
                    if cloudflared_proc.poll() is not None:
                        print(f"⚠️ Cloudflared process exited unexpectedly with code {cloudflared_proc.returncode}")
                        # Try to restart it
                        print("Attempting to restart cloudflared...")
                        try:
                            cloudflared_proc = subprocess.Popen(
                                ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
                                stdout=open(cloudflared_log, "a"),  # Append to existing log
                                stderr=subprocess.STDOUT
                            )
                            print("✅ Cloudflared restarted")
                        except Exception as e:
                            print(f"❌ Failed to restart cloudflared: {str(e)}")
                            break
                    
                    if streamlit_proc.poll() is not None:
                        print(f"⚠️ Streamlit process exited unexpectedly with code {streamlit_proc.returncode}")
                        break
                    
                    time.sleep(60)  # Check every minute
            except KeyboardInterrupt:
                print("Received keyboard interrupt. Shutting down...")
        else:
            print("\n⚠️ Could not verify connection after multiple attempts.")
            print("The tunnel might still be working. Please try the URL in your browser.")
    elif error_found:
        print("❌ Failed to establish Cloudflare tunnel due to errors.")
    else:
        print("❌ Timed out waiting for Cloudflare tunnel to establish.")

except Exception as e:
    print(f"❌ An unexpected error occurred: {str(e)}")

finally:
    # ALWAYS print the full content of the cloudflared log for debugging
    print("\n--- CLOUDFLARED LOG CONTENT ---")
    try:
        with open(cloudflared_log, "r") as f:
            print(f.read())
    except Exception as e:
        print(f"Could not read log file: {str(e)}")
    print("--- END OF LOG ---\n")
    
    # Only terminate processes if this is not a normal execution
    # (i.e., if we got here due to an error or manual interruption)
    if 'public_url' not in locals() or not public_url or not connection_success:
        print("Cleaning up processes...")
        
        # Clean up the cloudflared process
        if cloudflared_proc is not None:
            print("Terminating cloudflared process...")
            try:
                cloudflared_proc.terminate()
                cloudflared_proc.wait(timeout=5)
                print("✅ Cloudflared process terminated")
            except Exception as e:
                print(f"⚠️ Error terminating cloudflared: {str(e)}")
                try:
                    cloudflared_proc.kill()
                    print("✅ Cloudflared process killed")
                except:
                    pass
        
        # Clean up the streamlit process
        if 'streamlit_proc' in locals() and streamlit_proc is not None:
            print("Terminating streamlit process...")
            try:
                streamlit_proc.terminate()
                streamlit_proc.wait(timeout=5)
                print("✅ Streamlit process terminated")
            except Exception as e:
                print(f"⚠️ Error terminating streamlit: {str(e)}")
                try:
                    streamlit_proc.kill()
                    print("✅ Streamlit process killed")
                except:
                    pass

print("\nTroubleshooting tips:")
print("1. If you see 'command not found' errors, check if cloudflared was downloaded correctly.")
print("2. If you see connection errors, check your internet connection.")
print("3. You can try running the tunnel manually with:")
print("   ./cloudflared tunnel --url http://localhost:8501")
print("4. If all else fails, you can access the local version at: http://localhost:8501")